In [1]:
from deap import base, creator, tools
import random
import string
import itertools

signals = list(string.ascii_lowercase)  # All possible signals
meanings = list(range(0, 100))  # 100 possible meanings

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_word(min_length=1, max_length=3):
    length = random.randint(min_length, max_length)
    return ''.join(random.choice(signals) for _ in range(length))

def create_individual():
    individual = creator.Individual()
    individual.local_state = random.choice(meanings)
    for meaning in meanings:

        word = create_word()
        individual[meaning] = word

        word = create_word()
        individual[word] = meaning

    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)



In [2]:
GLOBAL_STATE = random.choice(meanings)

"""
One individual encodes a meaning into a signal space representation and
speaks to a listener. If that listener decodes the signal space representation
back into the original meaning, it is rewarded a fitness bonus. If the listener
does not have the percieved word in it's vocabularly, there is a small chance that it adopts it
and assigns it a random meaning. 
"""
def pairwise_communication(speaker, listener):
    fitness = 0
    local_encoded = speaker[speaker.local_state]
    global_encoded = speaker[GLOBAL_STATE]

    if local_encoded in listener:
        if listener[local_encoded] == speaker.local_state:
            fitness += 1
    elif (random.random() < .3): # add new word to dictionary, replace another. individual "learns" new word
        words = [key for key in listener if isinstance(key, str)]  # Get all words
        randomWord = random.choice(words)
        listener[local_encoded] = listener.pop(randomWord)
        listener[local_encoded] = random.choice(meanings)
        
    if global_encoded in listener:
        if listener[global_encoded] == GLOBAL_STATE:
            fitness += 1
    elif (random.random() < 1): # add new word to dictionary, replace another
        words = [key for key in listener if isinstance(key, str)]  # Get all words
        randomWord = random.choice(words)
        listener[global_encoded] = listener.pop(randomWord)
        listener[global_encoded] = random.choice(meanings)

    return fitness

def evaluate_group(group):
    for ind in group:
        ind.fitness.values = (0,)  # reset fitness

        # evaluate with self
        # for i in range(10):
        ind.local_state = random.choice(meanings)
        bonus_with_self = pairwise_communication(ind, ind)
        ind.fitness.values = (ind.fitness.values[0] + bonus_with_self,)
    
    for speaker, listener in itertools.permutations(group, 2):
        fitness_bonus = pairwise_communication(speaker, listener)
        speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)





In [3]:
population = toolbox.population(n=100)

GROUP_SIZE = 15

# eval groups
for i in range(0, len(population), GROUP_SIZE):
    group = population[i:i + GROUP_SIZE]
    evaluate_group(group)

for ind in population:
    #print(len(ind))
    print(ind, ind.fitness.values)

{0: 'j', 'ngt': 0, 1: 't', 'tfe': 1, 2: 'ki', 3: 'c', 'pym': 3, 4: 'b', 't': 18, 5: 'tw', 6: 'fg', 'dpf': 6, 7: 'o', 'ojs': 7, 8: 'tf', 9: 'uhv', 'xwu': 9, 10: 'nk', 'dkf': 10, 11: 't', 12: 'fit', 13: 'gj', 'q': 80, 14: 't', 'lvw': 14, 15: 'y', 'e': 16, 16: 'o', 17: 'e', 'maq': 17, 18: 'dgn', 19: 'hxl', 'opq': 19, 20: 'jwz', 'vg': 20, 21: 'kdk', 'yl': 21, 22: 'jtz', 'qq': 22, 23: 'u', 'ijt': 23, 24: 'sf', 'as': 24, 25: 'r', 'm': 77, 26: 'g', 27: 'q', 'je': 27, 28: 'li', 'z': 88, 29: 'rjq', 'k': 55, 30: 'ud', 'ek': 30, 31: 'x', 'nr': 31, 32: 'zn', 'cxa': 32, 33: 'a', 'ou': 33, 34: 'w', 35: 'jp', 'os': 35, 36: 'rxa', 'lto': 36, 37: 'dal', 'h': 39, 38: 'f', 'pwo': 38, 39: 'lep', 40: 'w', 'rfp': 40, 41: 'hln', 42: 'gh', 'dc': 42, 43: 'hq', 'gy': 50, 44: 'ayj', 'n': 93, 45: 'hzk', 'qw': 45, 46: 'yed', 'na': 46, 47: 'dpt', 'f': 73, 48: 'm', 'r': 48, 49: 'wna', 'wf': 49, 50: 'lpo', 51: 'utb', 'lhk': 51, 52: 'nz', 'qd': 52, 53: 'hz', 'l': 53, 54: 'a', 'neg': 54, 55: 'mul', 56: 'zx', 57: 'vye',

In [4]:
from deap import algorithms
import numpy

# Parameters
NUM_GENERATIONS = 10000
CXPB = 0.1  # Crossover probability
MUTPB = 0.0  # Mutation probability (seems better at 0 for now)
GROUP_SIZE = 10

def crossover(ind1, ind2, indpb):
    # copy individuals
    child1, child2 = toolbox.clone(ind1), toolbox.clone(ind2)

    # uniform crossover for transmitter genes (meanings to words)
    for meaning in meanings:
        if random.random() < indpb:
            # swap the words for the meaning
            child1[meaning], child2[meaning] = ind2[meaning], ind1[meaning]

    # crossover for receptor genes (words to meanings)

    shared_words = set(ind1.keys()).intersection(ind2.keys())
    for word in shared_words:
        if random.random() < indpb:
            # swap the meanings for the word
            child1[word], child2[word] = ind2[word], ind1[word]

    return child1, child2

def mutate(ind, indpb):
    # mutate transmitters
    for meaning in meanings:
        if random.random() < indpb:
            if random.random() < 0.01:
                ind[meaning] = create_word()
            else:
                randomMeaning = random.randrange(0, len(meanings))
                currentWord = ind[meaning]
                ind[meaning] = ind[randomMeaning]
                ind[randomMeaning] = currentWord

    # Mutate receptors (word to meaning)
    words = [key for key in ind if isinstance(key, str)]  # Get all words
    for word in words:
        if random.random() < indpb:
            if random.random() < .99:
                # change what meaning a word maps to
                ind[word] = random.choice(meanings)
            else:
                # change word in key
                new_word = create_word()
                ind[new_word] = ind.pop(word)

    return ind

toolbox.register("select", tools.selRoulette) # inspired by paper
toolbox.register("evaluate", evaluate_group)
#toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mate", crossover, indpb=0.5)
toolbox.register("mutate", mutate, indpb =.01)



stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

# ea
for gen in range(NUM_GENERATIONS):
    #GLOBAL_STATE = random.randint(0,3)
    GlOBAL_STATE = random.choice(meanings)
    for ind in population:
        ind.local_state = random.choice(meanings)

    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # mutation and crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # eval for invalid individuals
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    for group in [invalid_ind[i:i + GROUP_SIZE] for i in range(0, len(invalid_ind), GROUP_SIZE)]:
        evaluate_group(group)

    
    population[:] = offspring

    fits = [ind.fitness.values[0] for ind in population]

    length = len(population)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"Generation {gen}: Min {min(fits)}, Max {max(fits)}, Avg {mean}, Std {std}")

# print final pop
for ind in population:
    print(ind, ind.fitness.values)


Generation 0: Min 0.0, Max 1.0, Avg 0.94, Std 0.23748684174075835
Generation 1: Min 0.0, Max 1.0, Avg 0.96, Std 0.19595917942265423
Generation 2: Min 0.0, Max 1.0, Avg 0.94, Std 0.23748684174075835
Generation 3: Min 0.0, Max 4.0, Avg 1.02, Std 0.37363083384538803
Generation 4: Min 0.0, Max 4.0, Avg 1.13, Std 0.6731270311018569
Generation 5: Min 0.0, Max 4.0, Avg 1.44, Std 1.070700705145934
Generation 6: Min 0.0, Max 6.0, Avg 2.09, Std 1.5433405327405876
Generation 7: Min 0.0, Max 6.0, Avg 2.94, Std 1.5672906558772053
Generation 8: Min 1.0, Max 6.0, Avg 3.64, Std 1.2043255373859671
Generation 9: Min 1.0, Max 10.0, Avg 4.1, Std 1.3892443989449803
Generation 10: Min 1.0, Max 10.0, Avg 4.55, Std 1.7168284713389401
Generation 11: Min 1.0, Max 10.0, Avg 4.82, Std 2.065817029651948
Generation 12: Min 1.0, Max 10.0, Avg 5.54, Std 2.273411533356862
Generation 13: Min 1.0, Max 10.0, Avg 6.21, Std 2.2903929793814872
Generation 14: Min 0.0, Max 10.0, Avg 7.27, Std 2.6640382880131455
Generation 15:

KeyboardInterrupt: 

In [ ]:
oneToOnes = []
for ind in population:
    indCount = 0
    print(ind, ind.fitness.values)
    words = [key for key in ind if isinstance(key, str)]  # Get all words
    for meaning in meanings:
        if ind[meaning] in words:
            if ind[ind[meaning]] == meaning:
                indCount += 1
    oneToOnes.append(indCount / len(meanings))
sum = 0
count = 0
for num in oneToOnes:
    sum += num
    count += 1
print(sum/count)
    

{0: 'y', 'lqm': 0, 1: 'z', 'ka': 1, 2: 'pw', 'lzh': 2, 3: 'fds', 'hi': 3, 4: 'ftk', 'kdc': 4, 5: 'soh', 6: 'yjk', 7: 'yru', 8: 'w', 'xl': 8, 9: 'ze', 'g': 88, 10: 'z', 'p': 97, 11: 'w', 12: 'j', 13: 'oic', 'ru': 13, 14: 'zr', 15: 'hs', 'pmj': 15, 16: 'qwo', 'ylq': 16, 17: 'oy', 'a': 56, 18: 'y', 19: 'ozw', 'n': 46, 20: 'kn', 'x': 20, 21: 'k', 'qm': 21, 22: 'uh', 'xfq': 22, 23: 'y', 24: 'p', 'e': 24, 25: 'dvq', 'j': 85, 26: 'c', 27: 'n', 28: 'omq', 'm': 28, 29: 'jyf', 30: 'lkj', 31: 'zuc', 32: 'nnf', 33: 'umh', 34: 'bkb', 'rp': 34, 35: 'is', 'lu': 35, 36: 'dg', 'qyd': 36, 37: 'p', 38: 'pej', 'dlm': 38, 39: 's', 'ec': 39, 40: 'b', 'er': 40, 41: 'o', 'yh': 41, 42: 'hdg', 'b': 42, 43: 'krw', 'as': 43, 44: 'gql', 'fsw': 44, 45: 'oy', 'txu': 45, 46: 'p', 47: 'z', 'qyv': 47, 48: 'keu', 'gmk': 48, 49: 'j', 'frt': 49, 50: 'lqr', 51: 'ho', 'me': 51, 52: 'w', 53: 'xjf', 54: 'u', 55: 'soj', 'pr': 55, 56: 'v', 57: 'jud', 58: 'io', 'o': 69, 59: 'a', 'jj': 59, 60: 'kq', 61: 'p', 62: 'wn', 'if': 62, 6